# Update Items

### Import Libraries

In [325]:
import glob
import os
import json
import xml.etree.ElementTree as ET
import requests
from urllib import parse
import urllib.request
import configparser
from tqdm import tqdm
import time
import tqdm
import re
from unidecode import unidecode

### Get configuration file

In [2]:
config = configparser.ConfigParser()
config.sections()
config.read('../config/api.ini')
client_id = config.get('main', 'client_id')
client_secret = config.get('main', 'client_secret')
endpoint = config.get('main', 'endpoint')

### Connect to the Api

In [3]:
params = {
    'key_identity': client_id,
    'key_credential': client_secret
}

access_url = endpoint
print("Connected to:    ", access_url)

Connected to:     http://localhost:8080/api/


### Navigate the xml and converts it to json

In [388]:
dictionary = {}

for file in glob.glob("../input/*"):
    root = ET.parse(file).getroot()
    fileName = str(os.path.basename(file))
    
    for group in root.findall('family/group'):
        groupName = group.attrib['name']
        item = group.findall('item')
        
        for value in item:
            fieldName = value.attrib['name']         
            matchingName = groupName +"_"+ fieldName
            fieldValue = value.text
            
            #fix getty links 
            if "getty" in str(fieldValue) and 'subjectid=' in str(fieldValue) and '[' in str(fieldValue) and '[' in str(fieldValue):
                name  = fieldValue.split('[')[0].split(']')[0]
                url  = fieldValue.split('[')[1].split(']')[0]

                if  'subjectid=' in str(url):
                    url  = url.split('subjectid=')[1]
                    fieldValue = name + "; http://vocab.getty.edu/page/tgn/" +  url            

            cleanedText = str(fieldValue).replace("None", "").replace("\n", "").replace("\t", " ").replace('"', '').replace("'", "")
            
            # Creating/appending to a JSON structure
            if fileName not in dictionary:
                dictionary[fileName] = { matchingName :  [cleanedText] }
            else:
                if matchingName not in dictionary[fileName]:
                    dictionary[fileName][matchingName] = [cleanedText]
                else:
                    dictionary[fileName][matchingName].append(cleanedText)

In [7]:
print(dictionary['M_186_555'].keys())

dict_keys(['Objekt_Sammlungsobjekt-ID', 'Objekt_Primärabbildung', 'Objekt_Archivalien', 'Objekt_Gemälde Maße max. (H x B x T in cm)', 'Objekt_Gemälde Material', 'Objekt_Rahmen Maße max. (H x B x T in cm)', 'Objekt_Rahmen Material', 'Objekt_Format', 'Kurzbefund_Gemälde', 'Kurzbefund_Schmuckrahmen', 'Gemälde_Konservierungsbedarf', 'Gemälde_Restaurierungsbedarf', 'Gemälde_Anmerkung', 'Gemälde_Sicherungsbeklebung', 'Gemälde_Andere Sicherungsmaßnahmen', 'Gemälde_Sicherungsbedarf: anbringen, verbessern, ergänzen', 'Schmuckrahmen_Konservierungsbedarf', 'Schmuckrahmen_Restaurierungsbedarf', 'Schmuckrahmen_Anmerkung', 'Schmuckrahmen_Sicherungsbeklebung', 'Schmuckrahmen_Andere Sicherungsmaßnahmen', 'Schmuckrahmen_Sicherungsbedarf: anbringen, verbessern, ergänzen', 'Einrahmung_Dringender Verbesserungsbedarf', 'Einrahmung_Anmerkung', 'Rückseite_Rückseite', 'Lagerung_Art der Lagerung', 'Lagerung_Anmerkung', 'Lagerung_Verbesserungsbedarf', 'Halber Reinigungszustand_Zustand', 'Halber Reinigungszustan

### Import People

In [11]:
omekaTemplate =  { "o:resource_template": {"o:id": 6}}
omekaClass = {"o:resource_class": {"o:id": 546}}

### Define a function to import different categories each time 

In [385]:
def importCategory(category, propertyLabel, propertyId):

    for i in tqdm.tqdm(dictionary.keys()): 
        for cat in  dictionary[i][category]:
            if cat:
                # Preparing entries
                if '[' and ']' in cat:        
                    try:
                        link = re.search("(?P<url>http?://[^\s]+)", cat).group("url").replace(']','')
                        cat = cat.split('[')[0]                
                        identifier = { propertyLabel: [ { "type": "uri", "property_id": propertyId, "@id": link, 'o:label': cat } ]}
                    except:
                        continue
                else: 
                    identifier = { propertyLabel: [ { "type": "literal", "property_id": propertyId, "@value": cat } ]}

                # Check if it is already in database  
                url = access_url + 'items?property[0][property]='+str(propertyId)+'&property[0][type]=in&property[0][text]=' + urllib.parse.quote(cat)
                urlResponse = json.loads(urllib.request.urlopen(url).read().decode('utf-8'))

                if not urlResponse:
                    print('Inserting: ', cat )
                    postFields = json.dumps({ **omekaClass, **omekaTemplate, **identifier})

                    # Post Request
                    url = access_url + 'items/'
                    headers = {"Content-Type": "application/json"}
                    r = requests.post(url, data=postFields, params = params, headers=headers)
                    time.sleep(.25)


### Import Artist

In [387]:
importCategory('Künstler_Künstlername', 'crm:P1_is_identified_by', 1899)

 14%|█▎        | 87/641 [00:19<02:06,  4.38it/s]

Inserting:  Christian Friedrich Fues (?)


 30%|██▉       | 191/641 [00:42<01:15,  5.94it/s]

Inserting:  Hetsch, Philipp Friedrich 


 30%|███       | 194/641 [00:44<02:37,  2.84it/s]

...

 97%|█████████▋| 620/641 [03:03<00:05,  4.05it/s]

Inserting:  Monogrammist LH (?)


 98%|█████████▊| 631/641 [03:06<00:02,  3.77it/s]

Inserting:  Anthonie Palamedesz (1601-1673) 


100%|██████████| 641/641 [03:10<00:00,  3.37it/s]


### Import Depicted People

In [386]:
importCategory('Dargestellte Person_Dargestellte Person', 'crm:P1_is_identified_by', 1899)

  7%|▋         | 47/641 [00:02<00:51, 11.46it/s]

Inserting:  Gonzaga, Cecilia (1425 - 1451) 


  9%|▊         | 56/641 [00:03<01:24,  6.90it/s]

Inserting:  Thanner, Gottfridt (Gottfridt Thanner ist in den zeitgenössischen Quellen der Reichsstadt Augsburg nicht nachzuweisen)


 16%|█▌        | 102/641 [00:07<00:31, 16.96it/s]

...

 94%|█████████▍| 605/641 [00:46<00:22,  1.59it/s]

Inserting:  Katharina, Königin von Zypern (Caterina Cornero) 


 95%|█████████▍| 606/641 [00:47<00:27,  1.28it/s]

Inserting:  Piccolomini, Marbet (?)


100%|██████████| 641/641 [00:48<00:00, 13.19it/s]
